# INST326 — Week 10 Exercises: Data Models, Properties, and Special Methods (Library Management)

**Focus (Week 10 only):** Python data model & class design refinements — `@property` (get/set/delete) with validation, dataclass options (equality, ordering, `field` options, `__post_init__`), special methods (`__str__`, `__repr__`, `__len__`, `__iter__`, `__contains__`, `__eq__`, ordering), and lightweight container patterns. Optional: `functools.total_ordering` and `dataclasses.replace`.

**Out of scope (Week 11+):** multiple inheritance/mixins, advanced design patterns, descriptors beyond `@property`, metaclasses, concurrency, ORMs, complex type-system features, network I/O, or frameworks.


### Starter Scaffold (Week-10-safe)

Below is minimal starter code from prior weeks, lightly adapted for Week 10. You may extend it while staying within the Week 10 scope.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field, replace
from datetime import datetime, timedelta
from typing import Dict, Iterable, Iterator, Optional, List
from functools import total_ordering

# --- Exceptions (basic) ---
class LibraryError(Exception): ...
class DuplicateBookError(LibraryError): ...
class OverdueLoanError(LibraryError): ...

@dataclass(frozen=True)
class Author:
    last: str
    first: str

@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    _title: str = field(repr=False)
    _copies: int = field(default=1, repr=False, compare=False)
    author: Optional[Author] = field(default=None, repr=False, compare=False)

    def __post_init__(self):
        object.__setattr__(self, "isbn", self.isbn.replace('-', '').replace(' ', ''))
        normalized_title = self._title.strip().upper() if self._title is not None else ""
        if not normalized_title:
            raise ValueError("title cannot be empty")
        object.__setattr__(self, "_title", normalized_title)

    # Comparable by title then ISBN (Week 10: ordering/special methods)
    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self.title, self.isbn) < (other.title, other.isbn)

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn  # equality by identity key

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self.title!r}, copies={self._copies})"

    def __str__(self) -> str:
        return f"{self.title} [{self.isbn}]"

    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        stripped = value.strip()
        if not stripped:
            raise ValueError("title cannot be empty")
        self._title = stripped.upper()

    # Week 10: properties for validation
    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @copies.deleter
    def copies(self) -> None:
        # Reset to zero (logical 'remove from circulation')
        self._copies = 0

    @property
    def label(self) -> str:
        return f"{self.title} [{self.isbn}]"

    def __bool__(self) -> bool:
        return self._copies > 0

    def __hash__(self) -> int:
        return hash(self.isbn)

    def with_more_copies(self, n: int) -> Book:
        if n < 0:
            raise ValueError("Cannot add negative number of copies")
        return replace(self, _copies=self._copies + n)

@dataclass
class Member:
    member_id: str
    email: str

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def mark_returned(self) -> None:
        self.returned = True

class Catalog(Iterable[Book]):
    """Lightweight container with special methods (Week 10)."""
    def __init__(self):
        self._books: Dict[str, Book] = {}

    # Container-like dunder methods
    def __len__(self) -> int:
        return len(self._books)

    def __iter__(self) -> Iterator[Book]:
        # Iterate in title order for determinism
        return iter(sorted(self._books.values()))

    def __contains__(self, isbn: object) -> bool:
        # Allow 'in' checks by ISBN or Book
        if isinstance(isbn, Book):
            return isbn.isbn in self._books
        return isinstance(isbn, str) and (isbn in self._books)

    def __getitem__(self, isbn: str) -> Book:
        return self._books[isbn]

    def __delitem__(self, isbn: str) -> None:
        if isbn not in self._books:
            raise KeyError(f"ISBN {isbn} not found")
        del self._books[isbn]

    # API
    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN exists: {book.isbn}")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

    def remove_book(self, isbn: str) -> Optional[Book]:
        return self._books.pop(isbn, None)

    def find_by_title_prefix(self, prefix: str) -> list[Book]:
        prefix = prefix.strip().upper()
        return [book for book in self if book.title.startswith(prefix)]
    
    def to_list(self) -> list[Book]:
        return list(sorted(self._books.values()))

    def contains_title(self, needle: str) -> bool:
        needle = needle.strip().upper()
        for book in self._books.values():
            if book.title == needle:
                return True
        return False

class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> Loan:
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=14)
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        return loan

    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()


## 1) Property validation

Add an `@property` named `title` to `Book` that strips whitespace on set and raises `ValueError` if empty after stripping. Keep the existing attribute name (override dataclass behavior carefully).

In [ ]:
# Your code here
# Hint: define a private _title and redirect in __post_init__ if needed.

# Added the following code in the scaffolding of the notebook, in the Book class.

    # @property
    # def title(self) -> str:
        # return self._title

    # @title.setter
    # def title(self, value: str) -> None:
        # stripped = value.strip()
        # if not stripped:
            # raise ValueError("title cannot be empty")
        # self._title = stripped.upper()

## 2) Read-only computed property

Add a read-only property `label` on `Book` that returns `f"{title} [{isbn}]"`. Attempting to set `label` should raise `AttributeError`. Demonstrate usage.

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, in the Book class.

    # @property
    # def label(self) -> str:
        # return f"{self.title} [{self.isbn}]"

# Shown -> Attempting to set label will raise AttributeError. The first print statement should still execute, and output 'DUNE : MESSIAH [1969]'

b = Book("1-969","  Dune : Messiah  ",3)
print(b.label)
try:
    b.label = "a new value"
except AttributeError as e:
    print(f"Caught AttributeError as expected : {e}")

## 3) `__post_init__` data normalization

Use `__post_init__` on `Book` to normalize ISBN by stripping hyphens/spaces and uppercasing the title. Ensure validation runs on initial values too.

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, in the Book class.

    # def __post_init__(self):
        # object.__setattr__(self, "isbn", self.isbn.replace('-', '').replace(' ', ''))
        # normalized_title = self._title.strip().upper() if self._title is not None else ""
        # if not normalized_title:
            # raise ValueError("title cannot be empty")
        # object.__setattr__(self, "_title", normalized_title)

## 4) `dataclasses.field` options

Modify `Book` so `_copies` is excluded from comparisons (`compare=False`) and included in `__repr__` via a custom `__repr__`. Explain briefly in a comment why we avoid comparing by copies.

In [ ]:
# Your code here

# In the scaffolding of the notebook, the Book class should reflect that the copies attribute has compare set to False, and is included in __repr__ .

# We avoid comparing by copies since inventory count isn't part of a books identity. If we did, books with a matching ISBN could compare unequal
# since the inventory count of a certain book changes over time, and this comparision would be illogical since matching ISBN logically indicates this.

## 5) Ordering with `total_ordering`

Given `__eq__` (by ISBN) and `__lt__` (by title, then ISBN), verify that sorting a mixed list of `Book` objects orders by title. Show a quick sort demo.

In [ ]:
# Your code here

# Shown -> This quick sort demo verifies that sorting a mixed list of book objects will order them by title, since the mixed order of entry into the 
# list will be reordered into an alphapetical order correct order (outputs DARKLY DREAMING DEXTER, DUNE, ZEBRA TALES, ZEBRA TALES).

mixed_books = [
    Book("KD284","Zebra Tales",2),
    Book("NC239","Darkly Dreaming Dexter",1),
    Book("PS425","zebra tales",1),
    Book("XE314","Dune",4)
]
sorted_books = sorted(mixed_books)
print("Sorted by title:")
for b in sorted_books:
    print(b.title)

## 6) Container protocol methods

Implement `__delitem__(self, isbn: str)` on `Catalog` to remove a book or raise `KeyError` if missing. Show that `len(catalog)` updates accordingly.

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, in the Catalog class.

    # def __delitem__(self, isbn: str) -> None:
        # if isbn not in self._books:
            # raise KeyError(f"ISBN {isbn} not found")
        # del self._books[isbn]

# Shown -> len(catalog) updates accordingly since the first print statement will print a length of 2, while the second one prints a length of 1.

cat = Catalog()
cat.add_book(Book("1-222","AAA",1))
cat.add_book(Book("2-222","BBB",1))
print(cat._books)
print(len(cat))    
isbn_to_exit = "1-222".replace('-', '').replace(' ', '')
del cat[isbn_to_exit]
print(len(cat))   

# Shown -> __delitem__ will raise KeyError when attempts are made to delete a missing item.

try:
    del cat["nonexistent_isbn"]
except KeyError as e:
    print(f"Caught KeyError as expected : {e}")

## 7) Slicing/lookup convenience

Implement `find_by_title_prefix(self, prefix: str) -> list[Book]` on `Catalog` that returns books whose title starts with the prefix (case-insensitive).

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, in the Catalog class.

    # def find_by_title_prefix(self, prefix: str) -> list[Book]:
        # prefix = prefix.strip().upper()
        # return [book for book in self if book.title.startswith(prefix)]

# Shown -> find_by_title_prefix() correctly returns books with titles that start with a given prefix ; demo will output ['DUNE', 'DUNE : MESSIAH']

cat2 = Catalog()
cat2.add_book(Book("RS140", "DUNE", 2))
cat2.add_book(Book("BY389", "Dune : Messiah", 1))
cat2.add_book(Book("OG932", "Fire and Blood", 1))
matches = cat2.find_by_title_prefix("DUNE")
print([b.title for b in matches])

## 8) Immutability via `replace`

Using `dataclasses.replace`, show how to create a new `Book` with a different title while leaving the original unchanged. Explain in a comment when immutability is helpful.

In [ ]:
# Your code here

# Shown -> dataclasses.replace allows us to create a new book with a different title while leaving the original unchanged, since printing the title
# of the first book will print 'A GAME OF THRONES' while the printing the title of the second book will print'NEW TITLE'.

b1 = Book("WO311", "A Game Of Thrones", 3)
b2 = replace(b1, _title="New Title")
print(b1.title) 
print(b2.title)   

# Immutability is helpful when you need to use objects as keys in a dict/set, because preventing the key fields from being mutable means
# that bugs in hash tables caused by inappropiate mutation of a key field are prevented from ever occurring.

## 9) Rich string representations

Customize `__repr__` and `__str__` for `Book` (or confirm from scaffold) and explain in a short comment how they help during debugging vs. user display.

In [ ]:
# Your code here

# In the scaffolding of the notebook, __repr__ and __str__ are implemented as desired.

# __repr__ is mainly for helping developers during debugging since it returns a literal code snippet which could be used to recreate an object, 
# while __str__ shows an object in its full state (all attributes of the object, formatted) and is meant for user display.

## 10) Truthiness protocol

Define `__bool__(self)` on `Book` so that a book evaluates to `True` if `copies > 0` and `False` otherwise. Show a one-line `if book:` demo.

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, in the Book class.

    # def __bool__(self) -> bool:
        # return self._copies > 0

# Shown -> __bool__() is functional since the first book will trigger a 'Sold out' print, while the other book will trigger a 'Has copies' print.

b1 = Book("WA363","A Feast For Crows", 0)
b2 = Book("IX603","A Dance With Dragons", 1)
if b1:
    print("Has copies")
else:
    print("Sold out")

if b2:
    print("Has copies")

## 11) Hashability decision

Decide whether `Book` should be hashable based on ISBN. If yes, implement `__hash__` consistent with `__eq__`. If not, explain why and show how using `Book` as a `dict` key could be risky if mutable fields affect identity.

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, in the Book class.

    # def __hash__(self) -> int:
        # return hash(self.isbn)

## 12) In-place operations vs. new objects

Write a method `with_more_copies(self, n: int) -> Book` that returns a **new** `Book` with copies increased by `n` (do not mutate `self`). Show a quick before/after demonstration.

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, in the Book class.

    # def with_more_copies(self, n: int) -> Book:
        # if n < 0:
            # raise ValueError("Cannot add negative number of copies")
        # return replace(self, _copies=self._copies + n)

# Shown -> with_more_copies() is functional since printing the copies attribute of object 1 prints 2, doing so for object 2 prints 5 (see: 2+3 = 5)

b_old = Book("999","Increment",2)
b_new = b_old.with_more_copies(3)
print(b_old.copies)      # 2
print(b_new.copies)  # 5

## 13) Catalog iteration contract

Demonstrate that the provided `Catalog.__iter__` yields books in **sorted** order by title. Add an assertion-based test in a code cell.

In [ ]:
# Your code here

# Shown -> provided Catalog.__iter__ yields books in sorted order by title since the assert statement at the bottom of the cell raises no errors.

cat = Catalog()
cat.add_book(Book("PL805", "CC", 1))
cat.add_book(Book("GP625", "AA", 1))
cat.add_book(Book("VR237", "BB", 1))
titles_sorted = [b.title for b in cat]
assert titles_sorted == ["AA", "BB", "CC"]

## 14) Membership semantics

Show that `isbn in catalog` and `book in catalog` both work via `__contains__`. Add a short test cell to verify both cases.

In [ ]:
# Your code here

# Shown -> __contains__ successfully allows both isbn in catalog, book in catalog to function as valid tests since neither assert will raise errors.

cat = Catalog()
b1 = Book("OP648", "Introduction to Information Science", 1)
cat.add_book(b1)
assert "OP648" in cat
assert b1 in cat

## 15) Defensive copying in accessors

Add a method `to_list(self) -> list[Book]` to `Catalog` that returns a shallow copy of the book list (sorted). Explain why returning internal structures directly is risky.

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, in the Catalog class.

    # def to_list(self) -> list[Book]:
        # return list(sorted(self._books.values()))

# Returning internal structures / data directly without a helper method would be risky as it allows users  tomutate internal catalog unintentionally.

## 16) Lightweight value object

Create a small `@dataclass(frozen=True)` called `Author` with fields `last`, `first`. Add an optional `author: Author | None` to `Book` (default `None`). Show how `frozen=True` prevents later mutation and why that can be desirable for identity keys.

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, JUST ABOVE the Book class.

# @dataclass(frozen=True)
# class Author:
    # last: str
    # first: str

# Added the following code in the scaffolding of the notebook, in the attribute statements of the Book class.
# author: Optional[Author] = field(default=None, repr=False, compare=False)

# Shown -> # frozen=True prevents mutation of identity objects since the exception will be triggered. This outcome is desirable for identity objects
# since they are essential to the identity of another object (the author of a book shouldnt change after publishing).

author1 = Author(last="Orwell", first="George")
b = Book("RI992", "Animal Farm", 1, author=author1)
print(b.author) 
try:
    b.author.last = "New Last Name"
except Exception as e:
    print(f"Author is immutable, see error msg : {e}")

## 17) Custom containment by predicate

Implement `contains_title(self, needle: str) -> bool` on `Catalog` that returns `True` if any book title equals `needle` case-insensitively. Do not modify `__contains__`.

In [ ]:
# Your code here

# Added the following code in the scaffolding of the notebook, in the Catalog class.

    # def contains_title(self, needle: str) -> bool:
        # needle = needle.strip().upper()
        # for book in self._books.values():
            # if book.title == needle:
                # return True
        # return False

## 18) Pretty-print table helper

Write a standalone function `print_catalog_table(catalog: Catalog)` that prints a simple aligned table of ISBN, Title, Copies. Use only string formatting, no third-party libs.

In [ ]:
# Your code here

def print_catalog_table(catalog: Catalog) -> None:
    rows = [("ISBN", "Title", "Copies")]
    for b in catalog:
        rows.append((b.isbn, b.title, str(b.copies)))
    widths = [max(map(len, col)) for col in zip(*rows)]
    for row in rows:
        print(" | ".join(s.ljust(w) for s, w in zip(row, widths)))

# Shown -> print_catalog_table() is functional since the demo code below successfully prints a simple aligned table with ISBN, Title, Copies.        

cat = Catalog()
cat.add_book(Book("EI057", "Dune : Part One", 2))
cat.add_book(Book("YN467", "Dune : Part Two", 1))
print_catalog_table(cat)

## 19) Sorting with key functions

Demonstrate sorting books by different keys without changing their natural order: by ISBN, by copies (descending), by title length. Use `sorted(..., key=...)`.

In [ ]:
# Your code here

# Shown -> sorted(..., key=...) is used to output a books list in different orders of sorting (by different keys) without changing the natural order
# of the books list. The 3 print statements depict successful list sorting by ISBN ascending, copies descending, and shortest title first.

books = [
    Book("AA601", "Dune", 2),
    Book("AA342", "Dune : Messiah", 1),
    Book("AA573", "Dune : 2", 3)
]
by_isbn = sorted(books, key=lambda b: b.isbn)
by_copies_desc = sorted(books, key=lambda b: -b.copies)
by_title_len = sorted(books, key=lambda b: len(b.title))
print([b.isbn for b in by_isbn])         # ISBN ascending
print([b.isbn for b in by_copies_desc])  # Copies descending
print([b.isbn for b in by_title_len])    # Shortest title first

## 20) Minimal unit tests for data model

Using `unittest`, write a small test case verifying:
- `Book.title` setter strips and forbids empty
- `Book.copies` setter validates
- `Catalog` iteration is sorted
- `__contains__` works for both ISBN and Book
(Keep it basic—no fixtures beyond a simple `setUp`.)

In [ ]:
# Your code here
import unittest

# Shown -> Book.title setter strips and forbids empty successfully, Book.copies setter validates successfully, Catalog iteration is sorted 
# successfully, __contains__ works for both ISBN and Book successfully since the following unit tests will run without producing errors.

class TestWeek10DataModel(unittest.TestCase):
    def setUp(self):
        self.c = Catalog()
        self.c.add_book(Book("PI820","InSt327",1))
        self.c.add_book(Book("TP238","InSt326",2)) 
    def test_title_property(self):
        b = Book("SH648","  InSt314  ",1)
        self.assertEqual(b.title, "INST314")
        with self.assertRaises(ValueError):
            b.title = "  "
    def test_copies_validation(self):
        b = Book("NT257","INST346",1)
        b.copies = 2 
        self.assertEqual(b.copies, 2)
        with self.assertRaises(TypeError):
            b.copies = "x"
        with self.assertRaises(ValueError):
            b.copies = -2
    def test_sorted_iteration(self):
        titles = [b.title for b in self.c]
        self.assertEqual(titles, ["INST326", "INST327"])
    def test_contains(self):
        a = self.c.get_book("TP238")
        self.assertIn("TP238", self.c)
        self.assertIn(a, self.c)

# # To run in notebook:
unittest.main(argv=['-v'], exit=False)